# Lemma Files

In [156]:
import pandas as pd
import os

path = os.getcwd()
print(path)

jasist_path = path+"\\output\\lemma files\\jasist_cooccur\\"
scient_path = path+"\\output\\lemma files\\scientometrics_cooccur\\"
jasist_files = os.listdir(jasist_path)
scient_files = os.listdir(scient_path)

C:\Users\Liser\Desktop\linchengwei_link_prediction


## Read Files

In [157]:
df_lemma_nodes = []
df_lemma_edges = []
for f in scient_files:
    if f.endswith("nodes.pkl"):
        df_iter = pd.read_pickle(scient_path+f)
        df_lemma_nodes.append(df_iter)
    else:
        df_iter = pd.read_pickle(scient_path+f)
        df_lemma_edges.append(df_iter)

# Generate Cosin Similarity to do Vocab Control

In [168]:
path = os.getcwd()
scientometric_coauthor_path = path+'\\data\\Scientometrics-3\\co-author\\'
scientometric_cooccur_path = path+'\\data\\Scientometrics-3\\co-occurrance\\'
jasist_coauthor_path = path+'\\data\\JASIST\\co-author\\'
jasist_cooccur_path = path+'\\data\\JASIST\\co-occurrance\\'

def list_files_paths(files_path):
    files = os.listdir(files_path)
    network_file_paths = []
    mapfile_paths = []
    for file in files:
        print(files_path+file+'\\network.txt')
        print(files_path+file+'\\map.txt')
        network_file_paths.append(files_path+file+'\\network.txt')
        mapfile_paths.append(files_path+file+'\\map.txt')
    return files, network_file_paths, mapfile_paths

files, network_file_paths, mapfile_paths = list_files_paths(jasist_cooccur_path)

C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-occurrance\jasist_2010-2015\network.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-occurrance\jasist_2010-2015\map.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-occurrance\jasist_2010-2016\network.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-occurrance\jasist_2010-2016\map.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-occurrance\jasist_2010-2017\network.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-occurrance\jasist_2010-2017\map.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-occurrance\jasist_2010-2018\network.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-occurrance\jasist_2010-2018\map.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-occurrance\jasist_2010-2019\network.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST

In [169]:
def file_transform(file):
    source_list, target_list, weight_list = [], [], []
    with open(file, 'r', encoding='utf-8') as f:
        f = f.read()
        for i, line in enumerate(f.split('\n')):
            for j, ele in enumerate(line.split('\t')):
                if i == 0:
                    ele = ele.replace('\ufeff', '')
                if j == 0:
                    source_list.append(ele)
                elif j== 1:
                    target_list.append(ele)
                elif j == 2:
                    weight_list.append(ele)
    source_list = source_list[:-1]
    return source_list, target_list, weight_list

def to_df(source_list, target_list, weight_list):
    
    df = pd.DataFrame(columns = ['source', 'target', 'weight'])
    df['source'] = source_list
    df['target'] = target_list
    df['weight'] = weight_list

    return df

In [170]:
df_nodes = []
for f in mapfile_paths:
    df_iter = pd.read_csv(f, sep='\t')
    df_nodes.append(df_iter)

In [171]:
df_edges = []
for f in network_file_paths:
    source_list, target_list, weight_list = file_transform(f)
    df_iter = to_df(source_list, target_list, weight_list)
    df_edges.append(df_iter)

In [172]:
# all nodes, 2001-2010
def generate_node(df):
    N = list(set(list(set(df['source']))+list(set(df['target']))))
    nodes = []
    for n in N:
        nodes.append(int(n))
    nodes = sorted(nodes)
    print('num of nodes:{}'.format(len(nodes)))
    
    return nodes
node_list = []
for df in df_edges:
    nodes = generate_node(df)
    nodes = list(map(str, nodes))
    node_list.append(nodes)

num of nodes:255
num of nodes:364
num of nodes:378
num of nodes:378
num of nodes:350


In [173]:
import numpy as np
def create_empty_matrix(nodes):
    t = pd.DataFrame(np.zeros((len(nodes), len(nodes))))
    t.columns = nodes
    t.index = nodes
    return t

# t1_0 = create_empty_matrix(node_list[2]) # t1, direct
# t1_1 = create_empty_matrix(node_list[2]) # t1, cosine similarity
# t2_0 = create_empty_matrix(nodes) # t2, direct
# t2_1 = create_empty_matrix(nodes) # t2, cosine similarity

In [174]:
def add_weight(df, t, nodes):
    # add weight
    for i in nodes:
        # print(i)
        A = df['source']==i
        for k in range(len(df[A]['target'])):
            j = df[A]['target'].iloc[k]
            weight = df[A]['weight'].iloc[k]
            # print(i, j, weight)
            # print(df[A]['target'].iloc[k], df[A]['weight'].iloc[k])
            t.at[i, j] = int(weight)
            t.at[j, i] = int(weight)
    return t
# t1_0 = add_weight(df_edges[2], t1_0, node_list[2]) # t1, direct
# t2_0 = add_weight(df2_intersect, t2_0) # t2, direct

In [175]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
def generate_cosine_sim(t):
    t_sparse = sparse.csr_matrix(t)
    similarities = cosine_similarity(t_sparse)
    # remove self related
    def remove_self_related(similarities):
        for i in range(len(similarities)):
            for j in range(len(similarities)):
                if i==j:
                    similarities[i][j] = 0
        return similarities
    similarities = remove_self_related(similarities)
    
    return similarities

# t1_1 = generate_cosine_sim(t1_0)
# t2_1 = generate_cosine_sim(t2_0)

In [176]:
# df_t1_1_cosine = pd.DataFrame(t1_1)
def transform_to_df(similarities, intersect_n12):
    df = pd.DataFrame(similarities)
    df.columns = intersect_n12
    df.index = intersect_n12
    return df
# df_t1_1_cosine = transform_to_df(t1_1, node_list[2])

In [111]:
df_t1_1_cosine

,1,2,3,4,5,6,7,8,9,10,...,5961,5962,5963,5964,5965,5966,5967,5968,5969,5970
1,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.149071,0.0,0.000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.149071,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,...,0.105409,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000000,0.0,0.000000,0.0,0.875,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.102062,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5966,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
5967,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,...,0.000000,0.218218,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
5968,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
5969,0.0,0.000000,0.0,0.000000,0.0,0.000,0.0,0.2,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


# From Adjacency Matrix to Edge List

In [177]:
def from_adjacency_matrix_to_edge(df_t1_1_cosine):
    x = pd.DataFrame(df_t1_1_cosine.stack()).reset_index()
    x.columns = ['source', 'target', 'weight']
    x = x[x['weight']!=0]
    x = x.sort_values(by='weight', ascending=False)
    x.index=range(len(x))
    # x[0::2].to_pickle
    return x

In [100]:
# edge_index = 0
# edge_table = pd.DataFrame(columns=['source', 'target', 'weight'])
# edge_set = set()
# for source in node_list[1]:
#     arr = df_t1_1_cosine[df_t1_1_cosine[source]!=0]
#     target_list = list(arr.index)
#     # add source, target into edge set, see if existed already
#     for target in target_list:
#         A = (source, target) not in edge_set
#         B = (target, source) not in edge_set
#         if (A&B) :
#             edge_table.at[edge_index, 'source'] = source
#             edge_table.at[edge_index, 'target'] = target
#             edge_table.at[edge_index, 'weight'] = df_t1_1_cosine[source][target]
#             edge_set.add((source, target))
#             edge_set.add((target, source))
#             edge_index+=1

### main

In [179]:
for i, node_table in enumerate(node_list):
    print(i, "-"*10, '\n\tCreate Empty Matrix')
    t1_0 = create_empty_matrix(node_table) # t1, direct
    t1_1 = create_empty_matrix(node_table) # t1, cosine similarity
    print('\t Add Weight...')
    t1_0 = add_weight(df_edges[i], t1_0, node_table) # t1, direct
    print('\t Generate Cosine Similarity...')
    t1_1 = generate_cosine_sim(t1_0)
    print("\t node length:\t", len(t1_1))
    df_t1_1_cosine = pd.DataFrame(t1_1)
    print('\t Transform to dataframe')
    df_t1_1_cosine = transform_to_df(t1_1, node_table)
    print('\t From Adjacency Matrix to Edge...')
    edge_result = from_adjacency_matrix_to_edge(df_t1_1_cosine)
    edge_result[0::2].to_csv("{}_jasist_cooccur_cosine_similarity_edge.csv".format(files[i]))
    print('\tsaved')

0 ---------- 
	Create Empty Matrix
	 Add Weight...
	 Generate Cosine Similarity...
	 node length:	 255
	 Transform to dataframe
	 From Adjacency Matrix to Edge...
	saved
1 ---------- 
	Create Empty Matrix
	 Add Weight...
	 Generate Cosine Similarity...
	 node length:	 364
	 Transform to dataframe
	 From Adjacency Matrix to Edge...
	saved
2 ---------- 
	Create Empty Matrix
	 Add Weight...
	 Generate Cosine Similarity...
	 node length:	 378
	 Transform to dataframe
	 From Adjacency Matrix to Edge...
	saved
3 ---------- 
	Create Empty Matrix
	 Add Weight...
	 Generate Cosine Similarity...
	 node length:	 378
	 Transform to dataframe
	 From Adjacency Matrix to Edge...
	saved
4 ---------- 
	Create Empty Matrix
	 Add Weight...
	 Generate Cosine Similarity...
	 node length:	 350
	 Transform to dataframe
	 From Adjacency Matrix to Edge...
	saved


# Save

In [101]:
t = edge_table.sort_values(by='weight', ascending=False)

In [80]:
scientometrics_cooccur_2010-15

5

In [90]:
df_lemma_nodes[4].to_excel('scientometrics_cooccur_2010-19_lemma_nodes.xlsx')

In [102]:
t.to_pickle('scientometrics_cooccur_2010-16_cosine_similarity.pkl')

In [103]:
t[:1000].to_excel('scientometrics_cooccur_2010-16_cosine_similarity_top1000.xlsx')

In [104]:
t[:1000]

,source,target,weight
293513,1001,2222,1
540885,2318,4435,1
48243,195,217,1
153222,538,4011,1
251860,840,3169,0.92582
...,...,...,...
337721,1225,1261,0.833333
316573,1097,3186,0.833333
328003,1163,2379,0.833333
318550,1112,3948,0.833333
